In [12]:
import nltk
from nltk.corpus import brown
from nltk.tag import hmm
from sklearn.model_selection import train_test_split

# 1) Загрузим корпус и разобьём на train/test
nltk.download('brown')
nltk.download('universal_tagset')
tagged = brown.tagged_sents(tagset='universal')
train_sents, test_sents = train_test_split(tagged, test_size=0.2, random_state=42)

# 2) Соберём список всех тегов и слов из train
tags = list({t for sent in train_sents for (_, t) in sent})
words = list({w.lower() for sent in train_sents for (w, _) in sent})

# 3) Обучим HMM-теггер
trainer = hmm.HiddenMarkovModelTrainer(tags, words)
hmm_tagger = trainer.train_supervised(train_sents)

# 4) Оценим качество
print("Accuracy:", hmm_tagger.evaluate(test_sents))

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
C:\TMP\ipykernel_11296\1968036464.py:21: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print("Accuracy:", hmm_tagger.evaluate(test_sents))
c:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\tag\hmm.py:335: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
c:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\tag\hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
c:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\tag\hmm.py:363: Runtim

Accuracy: 0.7348217327003755
